Neural networks can be constructed using the torch.nn package.

Now that you had a glimpse of autograd, nn depends on autograd to define models and differentiate them. An nn.Module contains layers, and a method forward(input)that returns the output.

A typical training procedure for a neural network is as follows:

Define the neural network that has some learnable parameters (or weights)
Iterate over a dataset of inputs
Process input through the network
Compute the loss (how far is the output from being correct)
Propagate gradients back into the network’s parameters
Update the weights of the network, typically using a simple update rule: weight = weight - learning_rate * gradient

In [11]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class Net(nn.Module):
    
    def __init__(self):
        super(Net, self).__init__()
        # 1 input image channel, 6 output channels, 3x3 square convolution
        # kernel
        self.conv1 = nn.Conv2d(1, 6, 3)
        self.conv2 = nn.Conv2d(6, 16, 3)
        # an affine operation: y = Wx + b
        self.fc1 = nn.Linear(16 * 6 * 6, 120)  # 6*6 from image dimension
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        # Max pooling over a (2, 2) window
        x = F.max_pool2d(F.relu(self.conv1(x)), (2, 2))
        # If the size is a square you can only specify a single number
        x = F.max_pool2d(F.relu(self.conv2(x)), 2)
        x = x.view(-1, self.num_flat_features(x))
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

    def num_flat_features(self, x):
        size = x.size()[1:]  # all dimensions except the batch dimension
        num_features = 1
        for s in size:
            num_features *= s
        return num_features


In [12]:
net = Net()

In [13]:
type(net)

__main__.Net

In [14]:
class MyClass:
    """A simple example class"""
    i = 12345

    def f(self):
        return 'hello world'

In [16]:
english = MyClass()

In [17]:
english.i

12345

In [18]:
english.f

<bound method MyClass.f of <__main__.MyClass object at 0x7f44895aa860>>

In [19]:
english.f()

'hello world'

In [20]:
english.__doc__

'A simple example class'

In [23]:
class Complex:
    def __init__(self, realpart, imagpart):
        self.r = realpart
        self.i = imagpart

x = Complex(3.0, -4.5)
x.r, x.i

(3.0, -4.5)

In [24]:
print(net)

Net(
  (conv1): Conv2d(1, 6, kernel_size=(3, 3), stride=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(3, 3), stride=(1, 1))
  (fc1): Linear(in_features=576, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)


In [27]:
type(net)

__main__.Net

In [26]:
net.summary()

AttributeError: 'Net' object has no attribute 'summary'

You just have to define the forward function, and the backward function (where gradients are computed) is automatically defined for you using autograd. You can use any of the Tensor operations in the forward function. The learnable parameters of a model are returned by net.parameters()



In [28]:
params = list(net.parameters())
print(len(params))
print(params[0].size())  # conv1's .weight

10
torch.Size([6, 1, 3, 3])


In [29]:
params

[Parameter containing:
 tensor([[[[ 0.1820,  0.3204, -0.0876],
           [ 0.1302, -0.2102,  0.3298],
           [ 0.0571,  0.0613,  0.3190]]],
 
 
         [[[ 0.1820,  0.1108, -0.3210],
           [ 0.0065, -0.2209,  0.0598],
           [-0.3203, -0.1176, -0.3291]]],
 
 
         [[[-0.3268, -0.2073,  0.1530],
           [-0.1213,  0.0338,  0.3159],
           [ 0.0342, -0.2437,  0.3053]]],
 
 
         [[[ 0.1991,  0.3117,  0.2914],
           [ 0.1860,  0.0556,  0.2291],
           [-0.1520, -0.2864,  0.1950]]],
 
 
         [[[ 0.3260, -0.2041,  0.1493],
           [-0.0361, -0.0236,  0.3070],
           [ 0.0159,  0.1547,  0.0891]]],
 
 
         [[[ 0.0544, -0.0118,  0.3191],
           [-0.2764,  0.2337, -0.0414],
           [ 0.0918,  0.0845,  0.0688]]]], requires_grad=True),
 Parameter containing:
 tensor([ 0.3264, -0.1917,  0.1186,  0.1169, -0.2070, -0.0095],
        requires_grad=True),
 Parameter containing:
 tensor([[[[ 4.0744e-04,  3.1800e-02,  9.9920e-02],
           [

In [30]:
input = torch.randn(1, 1, 32, 32)
out = net(input)
print(out)

tensor([[ 0.1166, -0.0296,  0.0753, -0.0041, -0.0069,  0.0192, -0.0666,  0.0302,
         -0.0053,  0.0900]], grad_fn=<AddmmBackward>)


Zero the gradient buffers of all parameters and backprops with random gradients:

In [31]:
net.zero_grad()
out.backward(torch.randn(1, 10))

torch.nn only supports mini-batches. The entire torch.nn package only supports inputs that are a mini-batch of samples, and not a single sample.

For example, nn.Conv2d will take in a 4D Tensor of nSamples x nChannels x Height x Width.

If you have a single sample, just use input.unsqueeze(0) to add a fake batch dimension.

Recap:
torch.Tensor - A multi-dimensional array with support for autograd operations like backward(). Also holds the gradient w.r.t. the tensor.

nn.Module - Neural network module. Convenient way of encapsulating parameters, with helpers for moving them to GPU, exporting, loading, etc.

nn.Parameter - A kind of Tensor, that is automatically registered as a parameter when assigned as an attribute to a Module.

autograd.Function - Implements forward and backward definitions of an autograd operation. Every Tensor operation creates at least a single Function node that connects to functions that created a Tensor and encodes its history.

At this point, we covered:
* Defining a neural network
* Processing inputs and calling backward

Still Left:
* Computing the loss
* Updating the weights of the network

In [32]:
1+1

2

Loss Function
A loss function takes the (output, target) pair of inputs, and computes a value that estimates how far away the output is from the target.

There are several different loss functions under the nn package . A simple loss is: nn.MSELoss which computes the mean-squared error between the input and the target.

For example:

In [33]:
output = net(input)

In [34]:
output == out

tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1]], dtype=torch.uint8)

In [35]:
target = torch.randn(10) 

In [36]:
target.shape

torch.Size([10])

In [37]:
target = target.view(1, -1)

In [38]:
target.shape

torch.Size([1, 10])

In [39]:
criterion = nn.MSELoss()

loss = criterion(output, target)

In [40]:
print(loss)

tensor(0.9411, grad_fn=<MseLossBackward>)


Now, if you follow loss in the backward direction, using its .grad_fn attribute, you will see a graph of computations that looks like this:

input -> conv2d -> relu -> maxpool2d -> conv2d -> relu -> maxpool2d;
      -> view -> linear -> relu -> linear -> relu -> linear;
      -> MSELoss;
      -> loss;

So, when we call loss.backward(), the whole graph is differentiated w.r.t. the loss, and all Tensors in the graph that has requires_grad=True will have their .grad Tensor accumulated with the gradient.

For illustration, let us follow a few steps backward:

In [41]:
print(loss.grad_fn)  # MSELoss
print(loss.grad_fn.next_functions[0][0])  # Linear
print(loss.grad_fn.next_functions[0][0].next_functions[0][0])  # ReLU

In [43]:
net.zero_grad()     # zeroes the gradient buffers of all parameters

print('conv1.bias.grad before backward')
print(net.conv1.bias.grad)

loss.backward()

print('conv1.bias.grad after backward')
print(net.conv1.bias.grad)

conv1.bias.grad before backward
tensor([0., 0., 0., 0., 0., 0.])
conv1.bias.grad after backward
tensor([-0.0053,  0.0104,  0.0081,  0.0034, -0.0079,  0.0214])


The only thing left to learn is:

Updating the weights of the network

The simplest update rule used in practice is the Stochastic Gradient Descent (SGD):

weight = weight - learning_rate * gradient

We can implement this using simple python code:

In [44]:
learning_rate = 0.01
for f in net.parameters(): #interate over all the learnable parameters of net
    f.data.sub_(f.grad.data * learning_rate)

However, as you use neural networks, you want to use various different update rules such as SGD, Nesterov-SGD, Adam, RMSProp, etc. To enable this, we built a small package: torch.optim that implements all these methods. Using it is very simple:

In [45]:
import torch.optim as optim

# create your optimizer
optimizer = optim.SGD(net.parameters(), lr=0.01)

# in your training loop:
optimizer.zero_grad()   # zero the gradient buffers
output = net(input)
loss = criterion(output, target)
loss.backward()
optimizer.step()    # Does the update

Observe how gradient buffers had to be manually set to zero using optimizer.zero_grad(). This is because gradients are accumulated as explained in Backprop section.

In [46]:
output = net(input)
loss = criterion(output, target)

In [48]:
loss #loss is not small due to random input. 

tensor(0.8967, grad_fn=<MseLossBackward>)